In [ ]:
#Importing all the relevant libraries

#importing libraries
import math #mathematical operations
import numpy as np #linear algebra
import scipy.special as sp #scientific computing library containing bessel functions
import matplotlib.pyplot as plt #plotting library
import pandas as pd #data processing (can read CSVs and create tables, interacts with excel)
import folium #mapping tool
import os #operating system functionality

def rad(x): #creating a function to convert degrees into radians (all np trig functions use rads)
    y = np.radians(x)
    return y
def deg(x): #creating a function to convert degrees into radians (all np trig functions use rads)
    y = np.degrees(x)
    return y


In [ ]:
#Importing data from excel

# defining the working directory (where this python script and the excel document are)
os.chdir("C:\\Users\\George\\Documents\\University\\Year 4\\Project\\Python Version 1") 

#importing the excel input sheet
input_sheet = pd.read_excel('Project_Excel.xlsx', sheet_name='Inputs', header = None)

#extracting location and date info
lat = input_sheet.iloc[3, 3] #(90 to -90), equator is at 0, north pole at 90, south pole at -90
long = input_sheet.iloc[4, 3] #(180 to -180), prime meridian is at 0
day = input_sheet.iloc[5, 3] #(1 to 31)
month = input_sheet.iloc[6, 3] #(January to December)
year = input_sheet.iloc[7, 3] #(2000 to 2050)

In [ ]:
#Showing map of provided latitude and longitude

map = folium.Map(location=[lat , long], zoom_start = 4) #creating map centered at provided location with zoom level 4
folium.Marker([lat, long], popup='Your Location').add_to(map) #adding marker to map

map #displaying map

In [ ]:
#converting date into a number of seconds after 00:00 on January 1st 
#neglecting leap years, daylight savings and general progression of time year on year
#will consider a day to start at 00:00 
#365 days in a year, 24 hours in a day, 60 mins in an hour, 60 secs in a min -> 86400 secs in a day

day = input_sheet.iloc[5, 3] #extracting day from excel sheet [row,column]
month = input_sheet.iloc[6, 3] #extracting month from excel sheet [row,column]
year = input_sheet.iloc[7,3] #extracting year from excel sheet [row,column]

if month == "January": #converting month string into an integer
    month = 1
if month == "February":
    month = 2
if month == "March":
    month = 3
if month == "April":
    month = 4
if month == "May":
    month = 5
if month == "June":
    month = 6
if month == "July":
    month = 7
if month == "August":
    month = 8
if month == "September":
    month = 9    
if month == "October":
    month = 10    
if month == "November":
    month = 11
if month == "December":
    month = 12    
    
if month>0: #converting start of month into number of days since Jan 1st
    month_days = 0
if month>1:
    month_days = month_days + 31
if month>2:
    if year%4 != 0:
        month_days = month_days + 28
    if year%4 == 0:
        month_days = month_days +29
if month>3:
    month_days = month_days + 31
if month>4:
    month_days = month_days + 30
if month>5:
    month_days = month_days + 31
if month>6:
    month_days = month_days + 30
if month>7:
    month_days = month_days + 31 
if month>8:
    month_days = month_days + 31    
if month>9:
    month_days = month_days + 30
if month>10:
    month_days = month_days + 31
if month>11:
    month_days = month_days + 30

n = year - 2000
days = day + month_days + n*365 + int(n/4) #number of days since Jan 1st 2000 (including leap years in 2000, 2004, etc)

day_start = 86400 * (days - 1) #number of seconds after midnight Jan 1st 2000 the given day starts
day_end = (86400 * days) - 1 #number of seconds after midnight Jan 1st 2000 the given day ends 
print("the date is:", day,"/", month, "/", year)
print(lat, long)

# Parameters describing location of the Sun 
$t$ is an array containing each second of the day in question since midnight January 1st 2000

$M$ is an array containing the mean anomaly for the sun each second of the day in question

$v$ is an array containing the true anomaly for the sun for each second

$ecliptic\_long$ is an array containing the longitude of the sun each second calculated from $L$ and $M$

$ecliptic\_lat$ is a constant (float) containing the longitude of the sun each second, approx 0 for whole orbit

(angles are relative to the perihelion, mean refers to circular orbit approximation)

In [ ]:
#parameters describing location of the earth relative to the sun
t = np.linspace(day_start, day_end, 86400)
t_rel = t-43200 #43200 comes from time being calculated relative to J2000 (mid-day on Jan 1st 2000)

M0 = 357.5291
M1 = 1.140741065 * (10**-5)
M = (M0 +M1*t_rel)%360 

c1 = 1.9148
c2 = 0.0200
c3 = 0.0003
v = (M + c1*np.sin(rad(M)) + c2*np.sin(2*rad(M)) + c3*np.sin(3*rad(M)))

ecliptic_long = (v + 102.9373 + 180)%360
ecliptic_lat = 0


# Parameters describing location of the Sun in the sky
$a$ is an array containing the right ascension of the Sun each second of the day in question

$d$ is an array containing declination of the Sun

$sd$ is the sidereal time

$azimuth$ is an array containing the azimuth angle of the sun (location dependant), relative to North

$elevation$ is an array containing the elevation angle of the sun (location dependant), relative to the horizon


In [ ]:
# parameters describing location of the sun in the sky

a2 = -2.4657
a4 = 0.0529
a6 = -0.0014
a = (ecliptic_long + a2*np.sin(2*rad(ecliptic_long)) + a4*np.sin(4*rad(ecliptic_long)) + a6*np.sin(6*rad(ecliptic_long)))%360

d1 = 22.7908
d3 = 0.5991
d5 = 0.0492
d = (d1*np.sin(rad(ecliptic_long)) + d3*(np.sin(rad(ecliptic_long)))**3 + d5*(np.sin(rad(ecliptic_long)))**5)

sd0 = 280.1470
sd1 = 0.004178074
sd = (sd0 + sd1*t_rel + long)%360

azimuth = deg(np.arctan(np.sin(rad(sd-a))/(np.cos(rad(sd-a))*np.sin(rad(lat)) - np.tan(rad(d))*np.cos(rad(lat)))))
azimuth_corrected = (azimuth -180)%360
elevation = deg(np.arcsin(np.sin(rad(lat))*np.sin(rad(d)) + np.cos(rad(lat))*np.cos(rad(d))*np.cos(rad(sd-a))))

plt.plot(azimuth, "--")
plt.plot(azimuth_corrected)
plt.plot(elevation)